KNN design and implementation

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
from sklearn.model_selection import KFold

1

In [3]:
df_iris = pd.read_csv(
"https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
header=None,
)

2 and 3

In [5]:
X_iris, y_iris = df_iris.iloc[:, 0:4].values, df_iris.iloc[:, 4].values

In [6]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.2, random_state=1)

4, 5 and 6

In [7]:
class KNearestNeighbors:
    def __init__(self, k, distance_metric="euclidean", weights="uniform"):
        self.k = k
        self.X_train = None
        self.y_train = None
        self.weights = weights

        if distance_metric == 'manhattan':
            self.distance_metric = 'cityblock'
        else:
            self.distance_metric = distance_metric
    
    def fit(self, X, y):
        """
        Store the 'prior knowledge' of you model that will be used
        to predict new labels.
        :param X : input data points, ndarray, shape = (R,C).
        :param y : input labels, ndarray, shape = (R,).
        """
        self.X_train = X
        self.y_train = y
        return self

    def predict(self, X):
        """Run the KNN classification on X.
        :param X: input data points, ndarray, shape = (N,C).
        :return: labels : ndarray, shape = (N,).
        """
        # 1. Compute the distances between X and X_train
        distances = self.__distance(X)

        # 2. Find the k nearest neighbors
        k_nearest_neighbors = self.__nearest_neighbors(distances)

        # 3. Find the majority class among the k nearest neighbors
        return self.__majority_class(k_nearest_neighbors)


    def __distance(self, X):
        if self.weights == "distance":
            return 1 / cdist(X, self.X_train, metric=self.distance_metric)
        return cdist(X, self.X_train, metric=self.distance_metric)
    
    def __nearest_neighbors(self, distances):
        return np.argsort(distances, axis=1)[:, :self.k]
    
    def __majority_class(self, k_nearest_neighbors):
        return np.apply_along_axis(self.__counting_votes, axis=1, arr=self.y_train[k_nearest_neighbors])
    
    def __counting_votes(self, votes):
        labels, counts = np.unique(votes, return_counts=True)
        return labels[np.argmax(counts)]
        

7

In [8]:
y_pred_iris = KNearestNeighbors(k=5).fit(X_train_iris, y_train_iris).predict(X_test_iris)
y_pred_iris

array(['Iris-setosa', 'Iris-versic', 'Iris-versic', 'Iris-setosa',
       'Iris-virgin', 'Iris-versic', 'Iris-virgin', 'Iris-setosa',
       'Iris-setosa', 'Iris-virgin', 'Iris-versic', 'Iris-setosa',
       'Iris-virgin', 'Iris-versic', 'Iris-versic', 'Iris-setosa',
       'Iris-versic', 'Iris-versic', 'Iris-setosa', 'Iris-setosa',
       'Iris-versic', 'Iris-versic', 'Iris-versic', 'Iris-setosa',
       'Iris-virgin', 'Iris-versic', 'Iris-setosa', 'Iris-setosa',
       'Iris-versic', 'Iris-virgin'], dtype='<U11')

In [9]:
y_pred_iris == y_test_iris.astype('U11')

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [10]:
((y_pred_iris == y_test_iris.astype('U11')).all() == True)

True

9

In [56]:
df_mnist_full = pd.read_csv(
"https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/mnist_test.csv",
header=None,
)

In [58]:
df_mnist_sampled_per_number = [ df_mnist_full[df_mnist_full.iloc[:, 0] == i].sample(10, random_state=0) for i in range(10)]
df_mnist_sampled = pd.concat(df_mnist_sampled_per_number)


In [62]:
X_mnist, y_mnist = df_mnist_sampled.iloc[:, 1:].values, df_mnist_sampled.iloc[:, 0].values

In [63]:
X_train_mnist, X_test_mnist, y_train_mnist, y_test_mnist = train_test_split(X_mnist, y_mnist, test_size=0.2, random_state=1)

In [64]:
y_pred_mnist = KNearestNeighbors(k=5).fit(X_train_mnist, y_train_mnist).predict(X_test_mnist)
y_pred_mnist

array([4, 8, 3, 8, 9, 1, 1, 5, 5, 6, 9, 1, 1, 1, 1, 3, 3, 1, 1, 1],
      dtype=int64)

In [66]:
y_test_mnist

array([8, 8, 3, 8, 9, 1, 3, 8, 6, 6, 9, 3, 5, 5, 5, 3, 3, 4, 7, 1],
      dtype=int64)

In [65]:
((y_pred_mnist == y_test_mnist).all() == True)

False

In [68]:
((y_pred_mnist == y_pred_mnist).any() == False)

False

10

In [75]:
ks = list(range(1, 20, 2))
ks

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

KFold for iris

In [73]:
accuracies = []
for k in ks:
    accuracies_per_fold = []
    for train_index, test_index in KFold(n_splits=5).split(X_train_iris):
        X_train, X_test = X_train_iris[train_index], X_train_iris[test_index]
        y_train, y_test = y_train_iris[train_index], y_train_iris[test_index]
        y_pred = KNearestNeighbors(k=k).fit(X_train, y_train).predict(X_test)
        accuracies_per_fold.append(np.mean(y_pred == y_test))
    accuracies.append((np.mean(accuracies_per_fold), np.std(accuracies_per_fold)))
    print(f"K={k} - Accuracy: {accuracies[-1][0]} ± {accuracies[-1][1]}")

K=1 - Accuracy: 0.725 ± 0.19112241568632857
K=3 - Accuracy: 0.7250000000000001 ± 0.20682789409984764
K=5 - Accuracy: 0.7166666666666667 ± 0.19790570145063197
K=7 - Accuracy: 0.7500000000000001 ± 0.21889875894272828
K=9 - Accuracy: 0.7583333333333334 ± 0.19965247584518234
K=11 - Accuracy: 0.7583333333333334 ± 0.21311186420907371
K=13 - Accuracy: 0.7500000000000001 ± 0.2041241452319315
K=15 - Accuracy: 0.7500000000000001 ± 0.2041241452319315
K=17 - Accuracy: 0.7416666666666668 ± 0.20982797186690286
K=19 - Accuracy: 0.7333333333333334 ± 0.20172864061517007


KFold for MNIST

In [76]:
accuracies = []
for k in ks:
    accuracies_per_fold = []
    for train_index, test_index in KFold(n_splits=5).split(X_train_mnist):
        X_train, X_test = X_train_mnist[train_index], X_train_mnist[test_index]
        y_train, y_test = y_train_mnist[train_index], y_train_mnist[test_index]
        y_pred = KNearestNeighbors(k=k).fit(X_train, y_train).predict(X_test)
        accuracies_per_fold.append(np.mean(y_pred == y_test))
    accuracies.append((np.mean(accuracies_per_fold), np.std(accuracies_per_fold)))
    print(f"K={k} - Accuracy: {accuracies[-1][0]} ± {accuracies[-1][1]}")

K=1 - Accuracy: 0.7625 ± 0.09999999999999999
K=3 - Accuracy: 0.675 ± 0.12119199643540823
K=5 - Accuracy: 0.625 ± 0.11858541225631422
K=7 - Accuracy: 0.525 ± 0.1224744871391589
K=9 - Accuracy: 0.5375 ± 0.1403121520040228
K=11 - Accuracy: 0.5125 ± 0.1446979612848778
K=13 - Accuracy: 0.5 ± 0.11180339887498948
K=15 - Accuracy: 0.4625 ± 0.1159202311936963
K=17 - Accuracy: 0.425 ± 0.1
K=19 - Accuracy: 0.425 ± 0.1


# Test space

In [24]:
knn = KNearestNeighbors(k=3).fit(coords2, y)

NameError: name 'coords2' is not defined

In [41]:
coords1 = [(35.0456, -85.2672),
          (35.1174, -89.9711),
          (35.9728, -83.9422),
          (36.1667, -86.7833),
          (36.1473, -86.7770),
          (36.0544, -86.6694),]

coords2 = [(35.0456, -85.2672),
          (35.1174, -89.9711),
          (35.9728, -83.9422),
          (29.9728, -83.9422),
          (35.0457, -84.9711),]
distances = cdist(coords1, coords2, 'euclidean')
distances

array([[0.        , 4.70444794, 1.6171966 , 5.24298816, 0.29610002],
       [4.70444794, 0.        , 6.0892811 , 7.92556272, 5.00051406],
       [1.6171966 , 6.0892811 , 0.        , 6.        , 1.38497279],
       [1.88558331, 3.35605413, 2.84770898, 6.81441461, 2.13089414],
       [1.86902085, 3.35603469, 2.84016572, 6.79415494, 2.11537169],
       [1.72738018, 3.43208273, 2.7284205 , 6.66509403, 1.97527177]])

In [42]:
1/distances

C:\Users\mateo\AppData\Local\Temp\ipykernel_21852\1245022006.py:1: RuntimeWarning: divide by zero encountered in divide
  1/distances


array([[       inf, 0.21256479, 0.61835401, 0.19073093, 3.37723723],
       [0.21256479,        inf, 0.164223  , 0.12617401, 0.19997944],
       [0.61835401, 0.164223  ,        inf, 0.16666667, 0.72203585],
       [0.53033987, 0.29796897, 0.35115948, 0.14674775, 0.46928657],
       [0.53503951, 0.2979707 , 0.35209213, 0.14718534, 0.47273016],
       [0.57891136, 0.29136827, 0.36651242, 0.15003539, 0.50625945]])

In [4]:
y = np.array([0, 0, 1, 1, 1])

In [36]:
args = np.argsort(distances, axis=1)
args

array([[0, 4, 2, 1, 3],
       [1, 0, 4, 2, 3],
       [2, 4, 0, 3, 1],
       [0, 4, 2, 1, 3],
       [0, 4, 2, 1, 3],
       [0, 4, 2, 1, 3]], dtype=int64)

In [44]:
k_mas_cercanos = args[:, :3]
k_mas_cercanos

array([[0, 4, 2],
       [1, 0, 4],
       [2, 4, 0],
       [0, 4, 2],
       [0, 4, 2],
       [0, 4, 2]], dtype=int64)

In [75]:
y[k_mas_cercanos]

array([[0, 1, 1],
       [0, 0, 1],
       [1, 1, 0],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1]])

In [117]:
y = np.array([0, 0, 1, 1, 1])

In [126]:
def most_voted(votes):
    labels, counts = np.unique(votes, return_counts=True)
    return labels[np.argmax(counts)]
    
np.apply_along_axis(most_voted, axis=1, arr=y[k_mas_cercanos])

array([1, 0, 1, 1, 1, 1])

In [78]:
labels, counts = np.unique(y[k_mas_cercanos][0], return_counts=True)
labels, counts

(array([0, 1]), array([1, 2], dtype=int64))

In [79]:
most_voted = labels[counts.argmax()]

In [80]:
most_voted

1